In [1]:
import pandas as pd
import transformers
transformers.logging.set_verbosity_error()
from backend.evaluation.evaluation import Evaluator
from backend.pipeline.DBHandler import DBHandler

In [2]:
def get_ground_truth_data(gt_file="./FAQ.xlsx"):
    # Load ground truth data from a file
    data = pd.read_excel(gt_file)
    QA_list = []
    for i, row in data.iterrows():
        if i == 1:
            break
        relevant_chunks_id = row["relevant_chunks_id"].strip().split(",")
        QA_list.append((row["question"], row["answer"], relevant_chunks_id))
    return QA_list

In [3]:
ground_truth_data = get_ground_truth_data()
db_handler = DBHandler(org_id='maccabi', user_id='evaluator')
evaluator = Evaluator(db_handler)
results = evaluator.evaluate(ground_truth_data)

In [ ]:
pd.set_option('display.max_colwidth',1)
results.to_csv("results.csv")
results.head(10)